## Fitting a diagonal covariance Gaussian mixture model to text data

In a previous assignment, we explored k-means clustering for a high-dimensional Wikipedia dataset. We can also model this data with a mixture of Gaussians, though with increasing dimension we run into two important issues associated with using a full covariance matrix for each component.
 * Computational cost becomes prohibitive in high dimensions: score calculations have complexity cubic in the number of dimensions M if the Gaussian has a full covariance matrix.
 * A model with many parameters require more data: observe that a full covariance matrix for an M-dimensional Gaussian will have M(M+1)/2 parameters to fit. With the number of parameters growing roughly as the square of the dimension, it may quickly become impossible to find a sufficient amount of data to make good inferences.
 
Both of these issues are avoided if we require the covariance matrix of each component to be diagonal, as then it has only M parameters to fit and the score computation decomposes into M univariate score calculations. Recall from the lecture that the M-step for the full covariance is:

\begin{align*}
\hat{\Sigma}_k &= \frac{1}{N_k^{soft}} \sum_{i=1}^N r_{ik} (x_i-\hat{\mu}_k)(x_i - \hat{\mu}_k)^T
\end{align*}

Note that this is a square matrix with M rows and M columns, and the above equation implies that the (v, w) element is computed by

\begin{align*}
\hat{\Sigma}_{k, v, w} &= \frac{1}{N_k^{soft}} \sum_{i=1}^N r_{ik} (x_{iv}-\hat{\mu}_{kv})(x_{iw} - \hat{\mu}_{kw})
\end{align*}

When we assume that this is a diagonal matrix, then non-diagonal elements are assumed to be zero and we only need to compute each of the M elements along the diagonal independently using the following equation.

\begin{align*}
\hat{\sigma}^2_{k, v} &= \hat{\Sigma}_{k, v, v}  \\
&= \frac{1}{N_k^{soft}} \sum_{i=1}^N r_{ik} (x_{iv}-\hat{\mu}_{kv})^2
\end{align*}

In this section, we will use an EM implementation to fit a Gaussian mixture model with diagonal covariances to a subset of the Wikipedia dataset. The implementation uses the above equation to compute each variance term.

We'll begin by importing the dataset and coming up with a useful representation for each article. After running our algorithm on the data, we will explore the output to see whether we can give a meaningful interpretation to the fitted parameters in our model.

In [1]:
import graphlab
from scipy.sparse import csr_matrix
import numpy as np
from sklearn.preprocessing import normalize
from scipy.sparse import spdiags
from copy import deepcopy
from sklearn.metrics import pairwise_distances
import sys

## Load Wikipedia data and extract TF-IDF features

Load Wikipedia data and transform each of the first 5000 document into a TF-IDF representation.

In [2]:
wiki = graphlab.SFrame("E:\\Machine Learning\\U.W\\Cluster and Retrieval\\people_wiki.gl/").head(5000)
wiki["tf_idf"] = graphlab.text_analytics.tf_idf(wiki["text"])

This non-commercial license of GraphLab Create for academic use is assigned to lxn1021@gmail.com and will expire on November 18, 2019.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\Xiaoning\AppData\Local\Temp\graphlab_server_1555989643.log.0


In [3]:
def sframe_to_scipy(x, column_name):
    # Create triples of (row_id, feature_id, count).
    # 1. Add a row number.
    x = x.add_row_number()
    # 2. Stack will transform x to have a row for each unique (row, key) pair.
    x = x.stack(column_name, ["feature", "value"])
    
    # Map words into integers.
    f = graphlab.feature_engineering.OneHotEncoder(features=["feature"])
    # 1. Fit the transformer.
    f.fit(x)
    # 2. The transform takes "feature" column and adds a new column "feature_encoding".
    x = f.transform(x)
    # 3. Get the feature mapping.
    mapping = f["feature_encoding"]
    # 4. Get the feature id to use for each key.
    x["feature_id"] = x["encoded_features"].dict_keys().apply(lambda x: x[0])
    
    # Create numpy arrays that contain the data for the sparse matrix.
    i = np.array(x["id"])
    j = np.array(x["feature_id"])
    v = np.array(x["value"])
    width = x["id"].max() + 1
    height = x["feature_id"].max() + 1
    
    # Create a sparse matrix.
    mat = csr_matrix((v, (i, j)), shape=(width, height))
    
    
    return mat, mapping

In [4]:
tf_idf, map_index_to_word = sframe_to_scipy(wiki, "tf_idf")

In [5]:
tf_idf

<5000x100282 sparse matrix of type '<type 'numpy.float64'>'
	with 881415 stored elements in Compressed Sparse Row format>

In [6]:
map_index_to_word

feature,category,index
feature,conflating,0
feature,diamono,1
feature,bailouts,2
feature,electionruss,3
feature,maywoods,4
feature,feduring,5
feature,spiderbait,6
feature,mcin,7
feature,sumiswald,8
feature,quinta,9


As in the previous assignment, we will normalize each document's TF-IDF vector to be a unit vector.

In [7]:
tf_idf = normalize(tf_idf)

In [8]:
tf_idf

<5000x100282 sparse matrix of type '<type 'numpy.float64'>'
	with 881415 stored elements in Compressed Sparse Row format>

## EM in high dimensions

EM for high-dimensional data requires some special treatment:
* E step and M step must be vectorized as much as possible, as explicit loops are dreadfully slow in Python.
* All operations must be cast in terms of sparse matrix operations, to take advantage of computational savings enabled by sparsity of data.
* Initially, some words may be entirely absent from a cluster, causing the M step to produce zero mean and variance for those words.  This means any data point with one of those words will have 0 probability of being assigned to that cluster since the cluster allows for no variability (0 variance) around that count being 0 (0 mean). Since there is a small chance for those words to later appear in the cluster, we instead assign a small positive variance (~1e-10). Doing so also prevents numerical overflow.

**Initializing mean parameters using k-means**

Recall from the lectures that EM for Gaussian mixtures is very sensitive to the choice of initial means. With a bad initial set of means, EM may produce clusters that span a large area and are mostly overlapping. To eliminate such bad outcomes, we first produce a suitable set of initial means by using the cluster centers from running k-means. That is, we first run k-means and then take the final set of means from the converged solution as the initial means in our EM algorithm.

In [9]:
from sklearn.cluster import KMeans

In [16]:
np.random.seed(5)
num_clusters = 25

kmeans_model = KMeans(n_clusters=num_clusters, n_init=5, max_iter=400, random_state=1, n_jobs=1)
kmeans_model.fit(tf_idf)
centriods = kmeans_model.cluster_centers_
cluster_assignment = kmeans_model.labels_

means = [centroid for centroid in centriods]

In [17]:
len(means)

25

In [18]:
means

[array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.79942936e-04, 1.38140667e-04, 7.52519518e-05]),
 array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        2.23579792e-04, 1.31387094e-04, 8.53687044e-05]),
 array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        2.57794488e-04, 9.10226593e-05, 8.63707823e-05]),
 array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        2.15889678e-04, 1.00662936e-04, 9.09111720e-05]),
 array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.90336708e-04, 1.39094073e-04, 8.35359900e-05]),
 array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        2.34408669e-04, 8.79722226e-05, 9.68849648e-05]),
 array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        2.43089838e-04, 8.02849784e-05, 8.49266981e-05]),
 array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        2.07034151e-04, 1.40097040e-04, 9.10716403e-05]),
 array([0.00000000e+00, 0.00000000e+00, 0.000000

**Initializing cluster weights**

We will initialize each cluster weight to be the proportion of documents assigned to that cluster by k-means above.

In [19]:
num_docs = tf_idf.shape[0]
weights = []

for i in xrange(num_clusters):
    # Compute the number of data points assigned to cluster i:
    num_assigned = len(cluster_assignment[cluster_assignment==i])
    w = float(num_assigned)/num_docs
    weights.append(w)

**Initializing covariances**

To initialize our covariance parameters, we compute $\hat{\sigma}_{k, j}^2 = \sum_{i=1}^{N}(x_{i,j} - \hat{\mu}_{k, j})^2$ for each feature $j$.  For features with really tiny variances, we assign 1e-8 instead to prevent numerical instability. We do this computation in a vectorized fashion in the following code block.

In [20]:
def diag(array):
    n = len(array)
    
    return spdiags(array, 0, n, n)

In [21]:
covs = []

for i in xrange(num_clusters):
    member_rows = tf_idf[cluster_assignment==i]
    cov = (member_rows.multiply(member_rows) - 2*member_rows.dot(diag(means[i]))).sum(axis=0).A1 / member_rows.shape[0] \
            + means[i]**2
    cov[cov < 1e-8] = 1e-8
    covs.append(cov)

**Running EM**

Now that we have initialized all of our parameters, run EM.

In [22]:
def log_sum_exp(x, axis):
    x_max = np.max(x, axis=axis)
    if axis == 1:
        return x_max + np.log( np.sum(np.exp(x-x_max[:,np.newaxis]), axis=1) )
    else:
        return x_max + np.log( np.sum(np.exp(x-x_max), axis=0) )

In [23]:
def logpdf_diagonal_gaussian(x, mean, cov):
    n = x.shape[0]
    dim = x.shape[1]
    assert(dim == len(mean) and dim == len(cov))

    # multiply each i-th column of x by (1/(2*sigma_i)), where sigma_i is sqrt of variance of i-th variable.
    scaled_x = x.dot( diag(1./(2*np.sqrt(cov))) )
    # multiply each i-th entry of mean by (1/(2*sigma_i))
    scaled_mean = mean/(2*np.sqrt(cov))

    # sum of pairwise squared Eulidean distances gives SUM[(x_i - mean_i)^2/(2*sigma_i^2)]
    return -np.sum(np.log(np.sqrt(2*np.pi*cov))) - pairwise_distances(scaled_x, [scaled_mean], 'euclidean').flatten()**2

In [24]:
def EM_for_high_dimension(data, means, covs, weights, cov_smoothing=1e-5, maxiter=int(1e3), thresh=1e-4, verbose=False):
    n = data.shape[0]
    dim = data.shape[1]
    mu = deepcopy(means)
    sigma = deepcopy(covs)
    K = len(mu)
    weights = np.array(weights)
    
    l1 = None
    l1_trace = []
    
    for i in range(maxiter):
        # E-step: compute responsibilities
        logresp = np.zeros((n, K))    
        for k in xrange(K):
            logresp[:, k] = np.log(weights[k]) + logpdf_diagonal_gaussian(data, mu[k], sigma[k])
        l1_new = np.sum(log_sum_exp(logresp, axis=1))
        if verbose:
            print(l1_new)
        sys.stdout.flush()
        logresp -= np.vstack(log_sum_exp(logresp, axis=1))
        resp = np.exp(logresp)
        counts = np.sum(resp, axis=0)
        
        # M-step: update weights, means, covariances
        weights = counts/np.sum(counts)
        for k in range(K):
            mu[k] = (diag(resp[:, k]).dot(data)).sum(axis=0)/counts[k]
            mu[k] = mu[k].A1
            sigma[k] = diag(resp[:, k]).dot(data.multiply(data)-2*data.dot(diag(mu[k]))).sum(axis=0) \
                        + (mu[k]**2)*counts[k]
            sigma[k] = sigma[k].A1 / counts[k] + cov_smoothing*np.ones(dim)
        
        # Check for convergence in log-likelihood
        l1_trace.append(l1_new)
        if l1 is not None and (l1_new-l1) < thresh and l1_new > -np.inf:
            l1 = l1_new
            break
        else:
            l1 = l1_new
    
    out = {"weights": weights, "means": mu, "covs": sigma, "loglik": l1_trace, "resp": resp}
    
    
    return out

In [25]:
out = EM_for_high_dimension(tf_idf, means, covs, weights, cov_smoothing=1e-10)

In [26]:
out["loglik"]

[3879297479.366981, 4883345753.533131, 4883345753.533131]

## Interpret clustering results

In contrast to k-means, EM is able to explicitly model clusters of varying sizes and proportions. The relative magnitude of variances in the word dimensions tell us much about the nature of the clusters.

Write yourself a cluster visualizer as follows.  Examining each cluster's mean vector, list the 5 words with the largest mean values (5 most common words in the cluster). For each word, also include the associated variance parameter (diagonal element of the covariance matrix).

A sample output may be:
```
==========================================================
Cluster 0: Largest mean parameters in cluster 

Word        Mean        Variance    
football    1.08e-01    8.64e-03
season      5.80e-02    2.93e-03
club        4.48e-02    1.99e-03
league      3.94e-02    1.08e-03
played      3.83e-02    8.45e-04
...
```

In [28]:
def visualize_EM_clusters(tf_idf, means, covs, map_index_to_word):
    print("")
    print("============================================================")
    
    num_clusters = len(means)
    
    for c in xrange(num_clusters):
        print('Cluster {0:d}: Largest mean parameters in cluster '.format(c))
        print('\n{0: <12}{1: <12}{2: <12}'.format('Word', 'Mean', 'Variance'))
        
        # The k'th element of sorted_word_ids should be the index of the word 
        # that has the k'th-largest value in the cluster mean. Hint: Use np.argsort().
        sorted_word_ids = np.argsort(-means[c])

        for i in sorted_word_ids[:5]:
            print '{0: <12}{1:<10.2e}{2:10.2e}'.format(map_index_to_word['category'][i], 
                                                       means[c][i],
                                                       covs[c][i])
        print '\n=========================================================='

In [29]:
visualize_EM_clusters(tf_idf, out['means'], out['covs'], map_index_to_word)


Cluster 0: Largest mean parameters in cluster 

Word        Mean        Variance    
poetry      1.51e-01    1.90e-02
poems       6.33e-02    6.45e-03
poet        5.91e-02    6.36e-03
de          4.77e-02    8.72e-03
literary    4.68e-02    3.29e-03

Cluster 1: Largest mean parameters in cluster 

Word        Mean        Variance    
she         1.60e-01    4.59e-03
her         1.04e-01    3.20e-03
music       1.53e-02    1.04e-03
actress     1.52e-02    1.14e-03
show        1.27e-02    7.33e-04

Cluster 2: Largest mean parameters in cluster 

Word        Mean        Variance    
football    7.45e-02    4.57e-03
club        5.84e-02    2.55e-03
league      5.72e-02    2.83e-03
season      5.06e-02    2.35e-03
played      3.79e-02    9.46e-04

Cluster 3: Largest mean parameters in cluster 

Word        Mean        Variance    
district    5.56e-02    4.00e-03
republican  5.47e-02    4.55e-03
senate      5.04e-02    5.21e-03
democratic  3.72e-02    2.46e-03
house       3.65e-02    2.07e

**Q1: Select all the topics that have a cluster in the model created above.**

## Comparing to random initialization

Create variables for randomly initializing the EM algorithm. 

In [30]:
np.random.seed(5)
num_clusters = len(means)
num_docs, num_words = tf_idf.shape

random_means = []
random_covs = []
random_weights = []

for k in range(num_clusters):
    
    # Create a numpy array of length num_words with random normally distributed values.
    # Use the standard univariate normal distribution (mean 0, variance 1).
    mean = np.random.normal(0, 1, num_words)
    
    # Create a numpy array of length num_words with random values uniformly distributed between 1 and 5.
    cov = np.random.uniform(1,6,num_words)
    
    # Initially give each cluster equal weight.
    weight = 1
    
    random_means.append(mean)
    random_covs.append(cov)
    random_weights.append(weight)

In [31]:
out_random_init = EM_for_high_dimension(tf_idf, random_means, random_covs, random_weights, cov_smoothing=1e-5)

In [32]:
out_random_init['loglik']

[-793165403.6892343,
 2282407852.9796767,
 2362262754.3453236,
 2362514453.9992857,
 2362514453.9995394,
 2362514453.9995394]

**Q2: Try fitting EM with the random initial parameters you created above. What is the final loglikelihood that the algorithm converges to? Choose the range that contains this value.**

**Q3: Is the final loglikelihood larger or smaller than the final loglikelihood we obtained above when initializing EM with the results from running k-means?**

**Q4: For the above model, `out_random_init`, use the `visualize_EM_clusters` method you created above. Are the clusters more or less interpretable than the ones found after initializing using k-means?**

In [33]:
visualize_EM_clusters(tf_idf, out_random_init['means'], out_random_init['covs'], map_index_to_word)


Cluster 0: Largest mean parameters in cluster 

Word        Mean        Variance    
she         3.90e-02    5.42e-03
her         2.54e-02    2.14e-03
music       2.12e-02    2.34e-03
singapore   1.77e-02    5.52e-03
bbc         1.17e-02    1.83e-03

Cluster 1: Largest mean parameters in cluster 

Word        Mean        Variance    
she         1.63e-02    2.46e-03
he          1.35e-02    1.09e-04
music       1.16e-02    1.12e-03
university  1.06e-02    3.07e-04
her         1.03e-02    8.35e-04

Cluster 2: Largest mean parameters in cluster 

Word        Mean        Variance    
she         3.12e-02    3.56e-03
her         2.41e-02    2.52e-03
music       1.51e-02    1.44e-03
he          1.10e-02    1.16e-04
festival    1.07e-02    2.03e-03

Cluster 3: Largest mean parameters in cluster 

Word        Mean        Variance    
she         2.70e-02    3.39e-03
her         1.81e-02    1.56e-03
film        1.48e-02    2.16e-03
series      1.06e-02    5.52e-04
physics     1.05e-02    4.08e

**Note**: Random initialization may sometimes produce a superior fit than k-means initialization. We do not claim that random initialization is always worse. However, this section does illustrate that random initialization often produces much worse clustering than k-means counterpart. This is the reason why we provide the particular random seed (`np.random.seed(5)`).